<a href="https://colab.research.google.com/github/NikolayLenkovNikolaev/SAS-in-Clinical-Trial/blob/main/B_L4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analisi di studi controllati con outcome conteggio (eventi multipli).
# Il modello di Poisson

## Outcome conteggio: eventi multipli per singolo paziente
Count data is increasingly common in clinical research

Examples include the number of adverse events occurring during a follow-up period, the number of hospitalizations, the number of seizures in epileptics, etc.


https://support.sas.com/resources/papers/proceedings/proceedings/sugi26/p247-26.pdf

ex:
- 70 pazienti randomizzati ad uno di 2 tipi differenti di dialisi
- Ogni paziente ha un suo tempo di osservazione
- Eventi di interesse: nausea e ipotensione
- Dati: Dyalisis_AE.xlsx

# Confronto tra tassi di evento
C’è differenza nel tasso mensile di eventi osservati per i due tipi di dialisi?
```
proc means data=dyalisis_ae sum mean median var;
var count time;
class group;
run;
```

- trotale eventi 196
- totale mesi di osservazione (persone tempo)
  - 196/84 = 2.33 tasso mensile del gr1
  - 257/88 =2.92-tasso mensiledel gr2


- $H_0: \lambda_0=\lambda_1$
- $\lambda_1/\lambda_0= 1$
- $\lambda_1/\lambda_0: rate ratio RR$

Come verificare statisticamente questa ipotesi?
- Possiamo stimare l’errore standard del RR e valutare con un test Z o chiquadrato di Wald l’ipotesi che RR=1.
- Come stimare l’errore standard del RR?

## Modello di Poisson
- La distribuzione di Poisson è il modello probabilistico standard usato per
outcome di tipo conteggio.

- Il numero di eventi avversi **r** osservati in un singolo paziente può essere
determinata da una v.c. di Poisson:
$P(R=r|\lambda)= e^{-\lambda} \lambda^{r/r!}$ - $\lambda>0, r=0,1,2...$

- dove $\lambda$ e' il valore atteso del numero di eventi

- Possiamo collegare la media $\lambda$ al predittore lineare $\nu = \beta`.x$ da:
- $g(\lambda) = \nu$ con funzione link g, che garantisce che i valori previsti siano positivi (essendo dei conteggi di eventi, non possono infatti essere negativi):

$$log{\lambda} = \nu=\\beta`.x$$ - [nel ns.esempio $log{\lambda}= \alpha + \beta*Group$]

- Possiamo quindi utilizzare il metodo della massima verosimiglianza e
interpolare ai dati osservati il modello di regressione di Poisson

Utilizzand o la PROC GENMOD di SAS:

```
proc genmod data=dyalisis_ae;
class group;
model count=group / link=log dist=Poisson;
run;
```

- ```link=log``` funzione link che lega il valore atteso $\lambda$ al predittore lineare $\beta`.x$


## Interpretazione dei parametri

- $exp[\alpha]= exp[1.99]= 7.31$ - numero medio di eventi avversi nel gr2 (riferimento)
- $exp[\alpha + \beta]= exp[1.99-0.27]=5.58$ - numero di eventi avversi nel gr1

Il rapporto tra i due valori attesi e' uguale a : exp[-0.27]=0.76

Verificare l’ipotesi nulla: $\lambda(gr1)= \lambda(gr2)$ cioe il rapporto sia =1 , equivale a nerificare l'ipotesi nulla che $\beta$ sia pari a 0

Nel nostro caso , otteniamo un p-value associato a questa ipotesi pari a 0.0043 (basato sul test di Wald: $(\beta/SE)^2$), valore che ci permette di dire che le evidenza a favore dell'ipotesi nulla sono deboli, e che quindi il trattamento 1 ha un'incidenza di eventi avversi inferiore al trattamento 2.

## Modello di Poisson e parametro di offset
Nell’analisi precedente non abbiamo però tenuto conto del fatto che gli eventi si sono sviluppati in ogni soggetto in tempi di lunghezza diversi (ovviamente se osservo un soggetto per più mesi, avrò maggior probabilità di osservare eventi).

Vediamo ora come tenere conto dei diversi tempi di osservazione per ogni soggetto.

Il numero di eventi avversi $Y_{ij}$ osservati nell’ i-mo paziente nel j-mo mese di osservazione è assunto avere distribuzione di Poisson con media li (interpretabile ora come tasso mensile di eventi)

Il conteggio totale di eventi $C_i = \sum_{j=1}^{T_i} Y_{ij} $osservati per un paziente nei $T_i$ medi di osservazione e' quindi somma di $T_I$ variabili casuali di Poisson indipendenti tra loro, ognuna con la stessa media $\lambda_i$

Quindi: $C_i \sim P_0[\theta_i=T_i \lambda_i]

Il modello di regressione diventa quindi:

$$log{\theta_i}= log{T_i} +log{\lambda_i}= log{T_i} + \beta`.x_i$$ - [nel ns.esempio. $log{\theta_i = log{T_i} + \alpha + \beta* Group_i}$]

Il temine $log{T_i}$ viene inserito come variabile esplicativa con coefficiente costante uguale all'unita.

Variabili di questo tipo ,nella sintassi della PROC GENMOD di SAS , vengono deifinite come variabili OFSET, e vanno prima di tutto create nel dataset e poi esplicitate tra le opzioni del modello [offset=]

```
data dia1;
set dyalisis_ae;
logt=log(time);
run;

proc genmod data=dia1;
class group;
model count=group / offset=logt link=log dist=Poisson;
run;
```

Interpretazione dei parametri:

Modello $log\theta_i = log T_i + \alpha+\beta*Group_i$

con $\alpha+\beta*Group_i= log{\lambda_i}$

exp 1.07 = 2.92 → tasso *mensile* di eventi avversi nel gruppo 2 (riferimento)
exp 1.07 − 0.22 = 2.33 → tasso *mensile* di eventi avversi nel gruppo 1

l rapporto tra tassi è uguale a: $exp(-0.22) = 0.80(IC95%: exp(-0.41), exp(-0.039)= 0.66 - 0.96)$

Verificare l’ipotesi nulla: $\lambda(gr1)=\lambda(gr2)$ cioe il loro rapporto sia =1, equivale a verificar l'ipotesi nulla che $\beta$ sia pari a 0.

Nel nostro caso, otteniamo un p-value associato a questa ipotesi pari a 0.0179, valore che ci permette di dire che le evidenze a favore dell’ipotesi nulla sono deboli, e che quindi il trattamento 1 ha un’incidenza di eventi avversi inferiore al trattamento 2.


exp (1.07) = 2.92 → tasso *mesnile* di eventi avversi nel gruppo 2 (riferimento)
exp (1.07 - 0.22) = 2.33 → tasso *mensile* di eventi avversi nel gruppo 1

- 196/84= 2.33 tassso del gr1
- 257/88=2.92 tasso del gr2


## Regressione di Poisson: bontà di adattamento del modello

Overall performance of the fitted model can be measured y two different tests.

There is the Pearson $/X^2$ satistics

$X^2= \sum_{i=1}^n \frac{y_i - exp[X_i\hat{\beta}]}{exp[X_i \hat{\beta}]}$


and the deviance statistic

$D= 2.\sum_{i=1}^n [y_i .log(\frac{y_i}{exp[X_i \hat{\beta} - (y_i - exp[X_i.\hat{\beta}])]})]$

https://support.sas.com/resources/papers/proceedings/proceedings/sugi26/p247-26.pdf

The popular measures of the adequacy of the model fit are devianc and Pearson chi-square($\chi^2$). If statistical model is correct then both quantities are asymptotically distributed as $\chi^2$ statistics with n-p degrees of freedom. Where n is number of subjects and p is the number of fitted parameters (two in our case: intercept and regression coefficient for divice variable). Thus if the regression model is adequate, the expected value of both the deviance and Pearson Chi-Square is equal (or close) to n-p (both scaled deviance $\sim = 1$ or the scaled Pearson Chi-Squared: $\chi^2/df \sim = 1$), otherwise there could be dount about validity of the model.

modello probabilistico di Poisson sembra non adattarsi bene ai dati.

Il modello di regressione di Poisson assume che la media e varianza del
numero di eventi attesi nell’unità di tempo sia la stessa.

Se la statistica X2 / DF è superiore a 1, potrebbe esserci evidenza di
‘overdispersion’ (cioè la varianza è superiore alla media)

In presenza di overdispersion l’inferenza basata sul modello di Poisson
non è corretta.

Perche?

Mentre la stima puntuale dei beta è corretta (infatti abbiamo visto che il beta
stimato equivale alla differenza osservata tra i tassi) , la loro variabilità è
sottostimata: si otterranno quindi degli IC più stretti e dei p-value più piccoli, rispetto al livello di confidenza stabilito (es. 95%) o all’errore di primo tipo tollerato.


In altre parole, in caso di ipotesi nulla vera, si rifiuterà l’ipotesi nulla più
frequentemente rispetto a quanto definito dall’errore di primo tipo che si è
disposti tollerare.


Regressione di Poisson e overdispersion:

Overdispersion: it is possible to account for overdispersion with respect to the Poisson model by indroducing a dispersion parameter $\phi$ into the relationship between the variance and the mean.

$Var[Y_i] = \phi.\mu$

```
data dia1;
set dyalisis_ae;
logt=log(time);
run;
proc genmod data=dia1;
class group;
model count=group /offset=logt link=log dist=Poisson scale=P;
run;
```


**0.0948 * 1.3997 = 0.1327**

Introduzione alla regressione di Poisson
Petrie_Sabin_Poisson_Rate.pdf

Analisi di eventi ripetuti per soggetto con SAS
Analysis of Count Data Using the SAS system.pdf



## Esercizi:

Esercizio 1: simulazione overdispersion

Condurre uno studio di simulazione considerando 5000 trial clinici condotti in
pazienti sottoposti a dialisi.
Ogni trial clinico ha due bracci di studio, uno trattato con il farmaco sperimentale e l’altro, di controllo, trattato con placebo.

All’interno di ogni braccio vengono arruolati 100 soggetti dializzati e vengono
seguiti per 6 settimane.

L’outcome, numero di eventi avversi registrati in 6 settimane, è di tipo conteggio.

Si ipotizzi che il trattamento sperimentale e quello di controllo abbiano la stessa efficacia nel prevenire gli eventi avversi e che, in media, la popolazione di interesse sperimenti 3 eventi avversi nell’arco di 6 settimane.

A) Verificare che, in media, il numero medio di eventi avversi e la variabilità del numero di eventi avversi siano uguali.

Introdurre ora overdispersion nel processo di estrazione dati.

Perché ha senso introdurre overdispersion? Alcuni pazienti potrebbero avere una
tendenza a sviluppare più eventi avversi rispetto ad altri, indipendentemente
dall'effetto del trattamento. In questa situazione la variabilità è maggiore rispetto a quella attesa sotto ipotesi di distribuzione casuale degli eventi (Poisson). Si può, ad esempio, estrarre per ogni soggetto il parametro della v.c. di Poisson da una distribuzione esponenziale (che ha il vantaggio di essere sempre positiva). In questo modo a livello di popolazione si mantiene la stessa media di eventi avversi che avremmo ottenuto simulando senza overdispersion, con
variabilità però molto maggiore.

B) Fare nuovamente la verifica impostata al punto A).

C) Analizzare i dati simulati al punto B utilizzando un modello di Poisson non
corretto per overdispersion e valutare quanto vale l’errore di primo tipo reale.

D) Modificare lo studio di simulazione al fine di valutare come varia l’errore reale di primo tipo, rispetto all’errore nominale accettato (ipotizzare alpha=0.05), al crescere dell’intensità dell’overdispersion.

E) Verificare poi se il modello di Poisson corretto per overdispersion porta o meno ad un errore di primo tipo reale simile a quello nominale accettato.


02_PowerBySimulation_STATA.pdf


esercizio 3.2.:

Si ipotizzi di voler condurre uno studio pre-clinico per testare l’effetto della dose di un farmaco nei topi. La risposta di interesse (Y) è una variabile conteggio distribuita come una Poisson. Per un campione di 𝑛 topi, sia definito il modello di regressione di Poisson:

$y \sim Poisson(\mu)$ dove $log{\mu_i}= \beta_0 + \beta_1*x_i$

per i=1...n dove $x_i$ e' la dose im mg somministrata all'i-esimo topo.

I I ricercatori stanno cercando di decidere quale dimensione del campione
darebbe una potenza dell’80% di individuare un effetto della dose del farmaco $ \beta_1=0.65$ (ipotesi alternativa) rispetto all’ipotesi nulla di assenza di effetto $\beta_1=0$, , sotto un disegno sperimentale con tre livelli di dosaggio (0.2, 0.5 e 1.0
mg) ed un errore $\alpha$ del 5%. Si  ipotizzano gruppi di uguale numerosità, 𝑛 = 3 ∗ 𝑟. Definire il valore di 𝒓.

Risolvere l’esercizio impostando uno studio di simulazione su 1000 campioni e
fissando $\beta_0= 0$. (NB: nel modello considerare in continuo la variabile 𝑥 relativa ai livelli di dosaggio).

02_PowerBySimulation_STATA.pdf